In [1]:
import requests
import json
import os
import pandas as pd
import urllib.parse
from pathlib import Path
from datetime import date
from datetime import timedelta

# JSON PRINT HELPER #
def jprint(output):
    print(json.dumps(output, indent=4))

class ActionBuilderAPIClient:
    def __init__(self):
        # API #
        self.API_KEY = 'rD0a_l-Dlh_4zNa4o-vcKA'
        self.CUSTOM_AB_ENDPOINT = 'afttest'
        
        # HTTP REQUESTS #
        self.GET_HEADERS = {'OSDI-API-Token':self.API_KEY}
        self.API_URL = f'https://{self.CUSTOM_AB_ENDPOINT}.actionbuilder.org/api/rest/v1/'

        # INTERNAL #
        self.campaign_list = {}
        self.people_campaign_hash = {}
        self.people_worksite_hash = {}
        

    # GET and format json from requestURL
    def getJSON(self, url):
        r = requests.get(url = self.API_URL + url, headers = self.GET_HEADERS)
        return r.json()

    class ABuilderAFT:
    def __init__(self):

        # INTERNAL #
        self.APP_DICT = {}

        res = requests.get(url=self.LOADER_URL)
        objects = res.json()['application']['objects']

        for obj in objects:
            fields = {}
            name = obj['name']
            key = obj['key']
            
            if name in TABLES:
                for item in obj['fields']:
                    fields.update({item['name']:item['key']})
                self.APP_DICT.update({name:{'obj_id':key,'fields':fields}})
        
    # function to return key for any value
    def get_key(self, dictionary ,val):
        for key, value in dictionary.items():
            if val == value:
                return key

        return ''
        
    
    # GET and format json from requestURL
    def getJSON(self, url):
        r = requests.get(url = self.API_URL + url, headers = self.GET_HEADERS)
        return r.json()
    
    def getObjectJSON(self, object_name):
        return (self.getJSON('objects/' + self.APP_DICT[object_name]['obj_id']))['object']
        
    def find_matches(self, object_name, field_name, match_val):
        field_id = self.APP_DICT[object_name]['fields'][field_name]
        object_id = self.APP_DICT[object_name]['obj_id']
        
        match_filter = {'match':'and', 'rules':[{'field':field_id, 'operator':'is', 'value': match_val}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "objects/" + object_id + "/records?filters=" + filter_for_url
        res = self.getJSON(request_url)
        if res["total_records"] == 0:
            return ''
        else:
            return res["records"]
        
    def find_records_updated_at_date(self, ABuilder_object, date):
        #Convert to IDs
        ABuilder_object_id = self.APP_DICT[ABuilder_object]['obj_id']
        field_to_match_id = self.APP_DICT[ABuilder_object]['fields']['mdate']

        #Get Id
        match_filter = {'match':'and', 'rules':[{'field':field_to_match_id, 'operator':'is', 'value': date}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "https://api.ABuilder.aft.org/v1/objects/" + ABuilder_object_id + "/records?filters=" + filter_for_url

        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        if res_json_dict["total_records"] == 0:
            return ''
        else:
            return res_json_dict["records"]
        
    def get_identifier(self, ident):
        #Convert to IDs
        ABuilder_object_id = self.APP_DICT['obj_id']
        ABuilder_field_id = self.APP_DICT['fields']['ABuilderidentID']

        request_url = "https://api.ABuilder.aft.org/v1/objects/" + ABuilder_object_id + "/records/" +  ident
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        return res_json_dict[ABuilder_field_id]
    
    def get_connection(self,connection_name,field,connection):
        #Convert to IDs
        ABuilder_object_id = self.APP_DICT[connection_name]['obj_id']
        ABuilder_field_id = self.APP_DICT[connection_name]['fields'][field]

        request_url = "https://api.ABuilder.aft.org/v1/objects/" + ABuilder_object_id + "/connection_helper/" +  connection
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        return res_json_dict[ABuilder_field_id]

    
    
Overwrite:
    
Agency
Building
Campus
College
Department

Employer
Floor
Local Job Class
Program

Work Area
Work Region
Work Unit
Worksite

Chapter
Is Working?
Local

Local Dues Category
Unit
    
# MAIN LOGIC - WIP #
input_fields = [
    
                # PREFERRED_Name - to be added
    
                'Individualguid', #custom identifier in AB (just custom, not custom_X)
                'AffiliateNumber', #helps get campagin identfier
                'Lastname', #PI
                'Firstname', #PI
                'Middlename', #PI F, M, L

                'ChapterName', #chapter info tag
    
                'OfficerRoleName', #national officer role tag with note attched for local role
                    #"action_builder:note_response"
    
                'LocalJobClassName', #localjobclass tag
                'EmployerName', #Employer tag
    
                'CommitteeMemberType', #note attached to tag below
                'CommitteeType', #committe type tag 
    
                'JobTitleName', #create tag for Job title
                'JobDescription',#note attached to job class
    
                'isWorking', #respective tag
                'Department', #respective tag
                'Building', #respective tag
                'Campus', #respective tag
                'Agency', #respective tag
                'College', #respective tag
                'Program', #respective tag
                'WorkArea', #respective tag
                'WorkRegion', #respective tag
                'WorkUnit', #respective tag
                'WorkSite', #respective tag
                'Floor', #respective tag
    
                'RoomNumber', #TBD
    
                'WorkShift', #unknown
    
                'Unit', #unit tag
                'Localduescategoryname', #ldc tag
                'PaymentMethodName', #payment metod tag
                
                'UnionRelationship', #union relation tag
                #'start_date' #needs to be added as note to above tag, need field from query
    
                'Address', #PI
                'City', #PI
                'IndividualAddressTypeName', #not needed/ only send home address
                'State', #PI
                'Zip', #PI
    
                'Email1', #PI
                'Email2', #PI
                'EmailType1', #home/work
                'EmailType2', #same as above
                'Phone1', #PI
                'Phone2', #PI
                'PhoneType1', #home/work/mobile
                'PhoneType2' #PI


]


IndentationError: expected an indented block (Temp/ipykernel_15948/1300995592.py, line 36)

In [ ]:
individualguid
affiliatenumber
lastname
firstname
middlename
chaptername
officerrolename
localjobclassname
employername
committeemembertype
committeetype
jobdescription
isworking
department
building
campus
college
program
workarea
workregion
workunit
worksite
floor
unit
localduescategoryname
paymentmethodname
unionrelationship   
address
city
individualaddresstypename
state
zip
email1
email2
emailtype1
emailtype2  
phone1
phone2
phonetype1
phonetype2

In [ ]:
VIEW ONLY - Job Info - Do Not Edit + Department
VIEW ONLY - Job Info - Do Not Edit + Building
VIEW ONLY - Job Info - Do Not Edit + Local Job Class
VIEW ONLY - Job Info - Do Not Edit + Employer
VIEW ONLY - Job Info - Do Not Edit + Employer with Address
VIEW ONLY - Job Info - Do Not Edit + Worksite
VIEW ONLY - Job Info - Do Not Edit + Work Unit
VIEW ONLY - Job Info - Do Not Edit + Work Region
VIEW ONLY - Job Info - Do Not Edit + Work Area
VIEW ONLY - Job Info - Do Not Edit + Program
VIEW ONLY - Job Info - Do Not Edit + Floor
VIEW ONLY - Job Info - Do Not Edit + College
VIEW ONLY - Job Info - Do Not Edit + Campus

VIEW ONLY - Union Relationship Details - Do Not Edit + Local Dues Payment Method
VIEW ONLY - Union Relationship Details - Do Not Edit + Chapter
VIEW ONLY - Union Relationship Details - Do Not Edit + COPE Amount
VIEW ONLY - Union Relationship Details - Do Not Edit + Union Relationship
VIEW ONLY - Union Relationship Details - Do Not Edit + Unit (DO NOT EDIT)
VIEW ONLY - Union Relationship Details - Do Not Edit + Local Dues Categories
VIEW ONLY - Union Relationship Details - Do Not Edit + Is Working?
VIEW ONLY - Union Relationship Details - Do Not Edit + Local

Contact Status + Worksite Visit Status
Contact Status + House Visit Status
Contact Status + Phone Calls Contact Status
Contact Status + RSVP Status
Political Asks and Questions + 2020 Vote By Mail/GOTV General
Political Asks and Questions + 20 House District General
Political Asks and Questions + 20 School District General
Political Asks and Questions + 2020 Congressional District
Political Asks and Questions + 2020 Senate General
Political Asks and Questions + 20 Presidential General
Political Asks and Questions + 2020 AFT-Political Volunteer
Political Asks and Questions + 2020 Voter Registration
Political Asks and Questions + 2020 Governors General
 Support & Engagement + Meeting Attendance
 Support & Engagement + Took Action (Individual has Completed Action)
 Support & Engagement + Willing To (Shown interest, not Completed)
 Support & Engagement + Potential Leader
 Support & Engagement + Issues
 Support & Engagement + COPE (Enter dollar amount)
 Support & Engagement + Signed Membership Card
Union Role + Officer (Note the local role name)


In [ ]:
4 - Support and Engagement + Took Action (Individual has Completed Action)
6 - Union Relationship Details + Chapter
6 - Union Relationship Details + Local
5 - Job Info + Worksite
6 - Union Relationship Details + Unit
5 - Job Info + Local Job Class
5 - Job Info + Employer
5 - Job Info + Department
6 - Union Relationship Details + Local Dues Categories
7 - Union Role + Committee Type (Committee Member Type in Notes)
3 - Contact Status + RSVP Status
3 - Contact Status + Phone Calls Status
3 - Contact Status + Office / Workplace Visit Status
3 - Contact Status + House Visit Status
4 - Support and Engagement + Willing to... (Shown interest, not Completed)
4 - Support and Engagement + Potential Leader
4 - Support and Engagement + Meeting Attendance
4 - Support and Engagement + Issues
4 - Support and Engagement + COPE (Enter dollar amount)
4 - Support and Engagement + Become a Member
7 - Union Role + Officer (Make note of the local role name)
6 - Union Relationship Details + Union Relationship (DO NOT EDIT)
5 - Job Info + Work Unit
5 - Job Info + Work Region
5 - Job Info + Work Area
5 - Job Info + Program
6 - Union Relationship Details + Local Dues Payment Method
6 - Union Relationship Details + Is Working?

In [3]:
import requests
import json
import pandas as pd

INPUT_TABLES = ['actionbuilder_orgtx']
INPUT_SCHEMA = 'action_builder_api'

# JSON PRINT HELPER #
def jprint(output):
    print(json.dumps(output, indent=4))

class ActionBuilderAPIClient:
    def __init__(self):
        # API #
        self.API_KEY = 'qKAMp-xAALpfgzxuIe7RFA'
        self.CUSTOM_AB_ENDPOINT = 'aft'
        
        # HTTP REQUESTS #
        self.GET_HEADERS = {'OSDI-API-Token':self.API_KEY}
        self.POST_HEADERS = {'OSDI-API-Token':self.API_KEY, 'Content-Type': 'application/json'}
        self.API_URL = f'https://{self.CUSTOM_AB_ENDPOINT}.actionbuilder.org/api/rest/v1/'

        # INTERNAL #
        self.campaign_list = {}
        
    def loader(self):
        camps = self.getResource('campaigns')
        for camp in camps['_embedded']['action_builder:campaigns']:
            self.campaign_list.update({camp['name'].lower():camp['identifiers'][0]})
        
    # GET and format json from requestURL
    def getJSON(self, url):
        r = requests.get(url = self.API_URL + url, headers = self.GET_HEADERS)
        return r.json()
    
    def getResource(self, resource):
        return self.getJSON(resource)
        
client = ActionBuilderAPIClient()
client.loader()
jprint(client.campaign_list)

{
    "02048 - education austin": "action_builder:b5ea13f5-54a6-4720-b5a4-9c3cdd95d59a",
    "02048 - education austin [archive2]": "action_builder:987da2a0-9c96-4b2f-9f52-d509d5c20333",
    "orgtx - training": "action_builder:179ddc9a-288d-4426-9849-b4975e98118d",
    "00958 - providence teachers union": "action_builder:4aed1584-3428-469f-be3b-d82823aaae83",
    "md school for the deaf": "action_builder:7f6fe8d6-aa1c-43ec-9e22-c52297fa9f45",
    "00061 - uesf": "action_builder:f0e74f61-1e80-4b7b-beac-cac71dc480a1",
    "00527 - utno": "action_builder:d8dc4857-6cf1-4fea-a56d-6a95e3ab9619",
    "sjp - st. joe's prep organizing": "action_builder:d2011111-7369-45fd-9dad-5fa1047b8b09",
    "06448 - cy-fair aft (active)": "action_builder:3565950b-6f88-4521-a1f1-b687f67b0dcb",
    "08041 - texas aft": "action_builder:6cb50512-366d-4bc1-b0ef-47dc09be3667",
    "06198 - fort bend employees federation": "action_builder:4af11091-fad3-4a5c-bbe9-0a8073151195",
    "03456 - corpus christi aft": "ac